In [ ]:
!mkdir -p bert
!google-drive-ocamlfuse bert

In [ ]:
!pip install sentence-transformers

## Get embeddings PART1

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/GridData.csv')
df.shape

In [ ]:
df.head()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-chinese')#('distilbert-base-nli-mean-tokens')

In [ ]:
tdata=[]
for i in df['SUBTYPE'].values.tolist():
  tdata.append(str(i))

In [ ]:
embeddings = model.encode(tdata, show_progress_bar=True)

In [ ]:
import numpy as np
np.save('/content/bert/1976embeddings.npy',embeddings)

In [ ]:
import numpy as np
embeddings=np.load('/content/drive/MyDrive/1976embeddings.npy')

In [ ]:
emdeddings=np.load('/content/drive/MyDrive/embeddings.npy')
embeddings.shape

## UMAP Dimension reduction PART2

In [ ]:
import umap
umap_embeddings = umap.UMAP(n_neighbors=15,n_components=70,metric='cosine').fit_transform(embeddings)

In [ ]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=6,metric='euclidean',                      
cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
def get_umap_list(ID):
  return(umap_embeddings[ID])
df['umap']=df.apply(lambda x:get_umap_list(x['Doc_ID']),axis=1)

In [ ]:
umap_embeddings.shape

## cluster PART3

In [ ]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=6,metric='euclidean',                      
cluster_selection_method='eom').fit(umap_embeddings)
len(set(cluster.labels_))

## plot PART4

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# Prepare data
import pandas as pd
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(10, 5))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=1)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=2, cmap='hsv_r')
plt.colorbar()
# plt.savefig('/content/drive/MyDrive/bertword70_31.svg')
plt.savefig('/content/drive/MyDrive/bertword2021.svg')

In [ ]:
df['Topic'] = cluster.labels_

In [ ]:
df['Doc_ID'] = range(len(df))

In [ ]:
df=df.fillna("")

In [ ]:
docs_per_topic = df.groupby(['Topic'], as_index = False).agg({'SUBTYPE': ' '.join})

In [ ]:
len(set(df['Topic'].values.tolist()))

In [ ]:
df.head()

In [ ]:
set(df['Topic'].values.tolist())

In [ ]:
df.to_csv('/content/bert/bertword7.csv',index=None)